### Lang Chain Basics 

In [ ]:
#!pip install langchain
#!pip install langchain-openai
#!pip install -U langchain-ollamma

### Create Chat Ollama Model

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

### Call ollama model

In [ ]:
response = llm.invoke("What is the capital of Serbia? And Kosovo is part of Serbia !!!")
print(response.content)

In [ ]:
response = llm.invoke("What is the capital city of Serbia?. *Zagreb, *Belgrade, *Sarajevo. Can you reposne me in json format like answers and value if is true or false of this 3 choices?" )
print(response.content)

### Straming the reposne 

In [ ]:
for chuch in llm.stream("What is the capital of Serbia? And Kosovo is part of Serbia !!!"):
    print(chuch.content, end='', flush=True)

### Uderstanding Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("What is the capital of {country}?")
prompt = prompt_template.format(country="Serbia")  
response = llm.invoke(prompt)
print(response.content)


### LangChain  Chaining mechanism


In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("What is the capital of {country}?")
chain = prompt_template | llm
results = chain.invoke({'country':"Serbia"})
print(results.content)

### Chaining with Parsers   

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate.from_template("What is the role of {type} in Software Testing ? ")
chain = prompt_template | llm | StrOutputParser()
results = chain.invoke({'type':"AI"})
print(results)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt_template = PromptTemplate.from_template("What is the role of {type} in Software Testing? Please answer in JSON format")
chain = prompt_template | llm | JsonOutputParser()
results = chain.invoke({'type':"AI"})
print(results)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings

# 1. Dokumenti
documents = [
    "Serbia is a country located in Southeast Europe, on the Balkan Peninsula. Its capital is Belgrade.",
    "Greece is a country in Southern Europe. Its capital is Athens.",
    "France is a country in Western Europe. Its capital is Paris."
]

# 2. Split on chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
split_docs = splitter.create_documents(documents)

# 3. Embeddings sa posebnim embedding modelom
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

# 4. Chroma base
vectorstore = Chroma.from_documents(split_docs, embeddings, persist_directory="./chroma_db")
vectorstore.persist()

# 5. Query
query = "What is the capital of Serbia?"
retrieved_docs = vectorstore.similarity_search(query, k=2)

# 6. Prompt
prompt_template = PromptTemplate.from_template(
    "Use the following information to answer the question.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n"
    "Answer:"
)
context = "\n".join([doc.page_content for doc in retrieved_docs])

# 7. LLM (Chat)
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

# 8. Finalni upit
final_prompt = prompt_template.format(context=context, question=query)
response = llm.invoke(final_prompt)

print("Question:", query)
print("Answer:", response.content)
